# 0. Name & Z-ID
Matt Borek

z1951125

so many functions with the same name make me dizzy, polymorphism is not fun

# 2. Course Class

In [290]:
class Course():
    def __init__(self, department, course_number, course_name, section, credits, times):
        self.department = department
        self.course_number = course_number
        self.course_name = course_name
        self.section = section
        self.credits = credits
        self.times = times
        self.instructor = None
        self.students = set()
        
    def enroll(self, student):
        self.students.add(student)
    
    def drop(self, student):
        self.students.discard(student)
        
    def change_time(self, updated_times):
        self.times = updated_times
        
    def __str__(self):
        readable_times = ', ' .join([f"{day} {start}:00-{end}:00" for day, start, end in self.times])
        return f"{self.department} {self.course_number}: {self.course_name}, Section {self.section} ({self.credits} credits), Times: {readable_times}"

# 3. Schedule Class

In [291]:
class Schedule:
    def __init__(self, courses=None):
        self.courses = courses or []
        
    @property
    def credits(self):
        return sum(course.credits for course in self.courses)    
        
    def add_course(self, course):
        self.courses.append(course)
        
    def remove_course(self, course):
        self.courses.remove(course)
        
    def __add__(self, course):
        self.add_course(course)
        return self.courses
        
    def __sub__(self, course):
        self.remove_course(course)
        return self.courses

# 1. Academic Classes

In [292]:
class Academic:
    MAX_CREDITS = 0
    def __init__(self, campus_id, first_name, last_name):
        self.campus_id = campus_id
        self.first_name = first_name
        self.last_name = last_name
        self.max_credits = self.MAX_CREDITS
        self.schedule = Schedule()
        
    def add_course(self, course):
        if self.max_credits < (self.schedule.credits + course.credits):
            raise Exception(f"Requested courseload would exceed maximum of {self.max_credits} credits")
        else:
            self.schedule.add_course(course)
    
    def remove_course(self, course):
        self.schedule.courses.remove(course)
        
    def __add__(self, course):
        self.add_course(course)
        return self
            
    def __sub__(self, course):
        self.remove_course(course)
        return self

In [293]:
class Student(Academic):
    MAX_CREDITS = 16
    def __init__(self, campus_id, first_name, last_name, level):
        super().__init__(campus_id, first_name, last_name)
        self.level = level
        
    def add_course(self, course):
        super().add_course(course)
        course.enroll(self)
    
    def remove_course(self, course):
        super().remove_course(course)
        course.drop(self)

In [294]:
class GraduateStudent(Student):
    MAX_CREDITS = 12
    def __init__(self, campus_id, first_name, last_name):
        super().__init__(campus_id, first_name, last_name, "Graduate")

In [295]:
class Instructor(Academic):
    MAX_CREDITS = 9
    def __init__(self, campus_id, first_name, last_name, rank):
        super().__init__(campus_id, first_name, last_name)
        self.rank = rank
        
    def add_course(self, course):
        super().add_course(course)
        course.instructor = self
        
    def remove_course(self, course):
        super().remove_course(course)
        course.instructor = None

# 4. Registrar Class

In [296]:
class Registrar:
    def __init__(self):
        self.courses = {}
        self.academics = {}
        
    def add_persons(self, persons):
        for person in persons:
            self.academics[person.campus_id] = person
            
    def add_courses(self, courses):
        for course in courses:
            self.courses[(course.department, course.course_number, course.section)] = course

    def add_person_to_course(self, campus_id, department, course_number, section):
        self.academics[campus_id].add_course(self.courses[(department, course_number, section)])

    def remove_person_from_course(self, campus_id, department, course_number, section):
        self.academics[campus_id].remove_course(self.courses[(department, course_number, section)])
        
    def print_schedule(self, campus_id):
        print(f"{campus_id}'s Schedule:")
        for course in self.academics[campus_id].schedule.courses:
            professor = course.instructor.last_name if course.instructor else "N/A"
            print(f"{course.__str__()} with Professor {professor}") 
            
    def print_enrollments(self, department, course_number, section):
        session = self.courses[(department, course_number, section)]
        print(f"{department} {course_number}({section}): {session.course_name} Enrollment")
        for student in session.students:
            print(student.first_name, student.last_name)

# Testing

In [297]:
s1 = Student("z143", "Catherine", "Smith", "Senior")
s2 = Student("z352", "Niraj", "Kumar", "Sophomore")
s3 = GraduateStudent("z785", "Divya", "Bharti")
s4 = GraduateStudent("z982", "James", "O'Brien")

i1 = Instructor("a421", "Jennifer", "Martinez", "Professor")
i2 = Instructor("a572", "Jonathan", "Jones", "Instructor")

c1 = Course("CSCI", 1543, "Programming Principles in Python", 1, 3, [("Mon", 10, 12), ("Wed", 10, 12)])
c2 = Course("CSCI", 1342, "Computer Networks", 2, 4, [("Tue", 14, 16), ("Thu", 14, 16), ("Fri", 12, 13)])
c3 = Course("CSCI", 1352, "Computer Graphics", 1, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c4 = Course("SOCI", 1230, "Introduction to Sociology", 1, 3, [("Mon", 11, 13), ("Thu", 11, 13)])
c5 = Course("POLS", 1100, "American Politics", 2, 3, [("Tue", 10, 12), ("Thu", 10, 12)])
c6 = Course("SOCI", 1450, "Classical Sociological Theory", 1, 3, [("Mon", 12, 13), ("Wed", 12, 13), ("Fri", 12, 13)])

r = Registrar()
r.add_persons([s1,s2,s3,s4,i1,i2])
r.add_courses([c1,c2,c3,c4,c5, c6])
r.add_person_to_course("a572", "SOCI", 1230, 1)
r.add_person_to_course("a572", "POLS", 1100, 2) 
r.add_person_to_course("a572", "SOCI", 1450, 1)
r.add_person_to_course("a421", "CSCI", 1543, 1)
r.add_person_to_course("a421", "CSCI", 1342, 2)
#r.add_person_to_course("a421", "CSCI", 1352, 1) # error due to max credits
r.add_person_to_course("z785", "SOCI", 1230, 1)
r.add_person_to_course("z785", "CSCI", 1352, 1)
r.add_person_to_course("z143", "CSCI", 1543, 1)
r.add_person_to_course("z143", "CSCI", 1342, 2)
r.add_person_to_course("z143", "SOCI", 1230, 1) 

r.print_schedule("z143") #Extra Credit
r.print_enrollments("SOCI", 1230, 1) #Extra Credit

z143's Schedule:
CSCI 1543: Programming Principles in Python, Section 1 (3 credits), Times: Mon 10:00-12:00, Wed 10:00-12:00 with Professor Martinez
CSCI 1342: Computer Networks, Section 2 (4 credits), Times: Tue 14:00-16:00, Thu 14:00-16:00, Fri 12:00-13:00 with Professor Martinez
SOCI 1230: Introduction to Sociology, Section 1 (3 credits), Times: Mon 11:00-13:00, Thu 11:00-13:00 with Professor Jones
SOCI 1230(1): Introduction to Sociology Enrollment
Divya Bharti
Catherine Smith
